In [5]:
# Import Python Libraries
import geemap
import ee
import pandas as pd
import geopandas as gdp
import json
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap      # Used for NDCI index bar

In [6]:
ee.Authenticate()                      # Initialize Earth Engine server for image collection
ee.Initialize(project='ee-jlzamary')   # change project name to yours (your GEE username)

In [ ]:
# Load data into a GeoDataFrame (gdf)
gdf = gdp.read_file('LagoonsPolygon.geojson') # Make sure to check file location 
                                              # should be a geojson file with coordinates

# Convert gdf to a GeoJSON stirng to include geometry and attribute data
geojson_str = gdf.to_json()

# Create the feature class (lagoon_fc)
lagoon_fc = ee.FeatureCollection(json.loads(geojson_str))

# Function to calculate Normalized Difference Chlorophyll Index
def calculate_ndci(image):
    '''
    Calculates the Normalized Difference Chlorophyll Index (NDCI).

    The NDCI is computed using band B5 (Red Edge) and band B4 (Red). NDCI is being used 
    to estimate chlorophyll-a concentrations in water bodies. Chlorophyll-a is a good indicator
    of agal blooms.

    Input:
        image (ee.Image): A satellite image (from Sentinel-2) containing bands B4 and B5.

    Output:
        ee.Image: The original image with an added 'NDCI' band.
    '''

    # Define and calculate NDCI bands
    ndci = image.normalizedDifference(['B5','B4']).rename('NDCI')

    # Add NDCI bands 
    return image.addBands(ndci)

# Function to mask land (only maintain bodies of water)
def mask_land(image):
    '''
    Masks out land areas from a satellite image using the Normalized Difference Water Index (NDWI).

    This function calculates the NDWI using bands B3 (Green) and B8 (Near-Infrared),
    then creates a mask to retain only pixels with NDWI values greater than 0.1,
    which typically correspond to water bodies. The resulting image contains only
    water pixels. In this case only the select lagoon should be diplayed

    Input:
        image (ee.Image): A satellite image (e.g., from Sentinel-2) with bands B3 and B8.

    Output:
        ee.Image: The input image with only water pixels
    '''
    # Define and calculate NDWI bands 
    ndwi = image.normalizedDifference(['B3','B8']).rename('NWDI')

    # Returns values only greater than 0.1 which typically is just water
    water_mask = ndwi.gt(0.1)

    # return the satellite image with just water
    return image.updateMask(water_mask)

# Convert Pandas Series to a Python list (allows for names to be printed)
lagoon_names = gdf['lagoon'].tolist()

# Print a list of lagoon names
print('Avalible lagoons:', lagoon_names)

# Ask user to pick a lagoon for data anlysis
lagoon_name = input('Enter an avlible lagoon: ')

# Filter data to selected lagoon
selected_feature = lagoon_fc.filter(ee.Filter.eq('lagoon', lagoon_name))

# For improper name input
if selected_feature == 0:
    print('Feature Class not Found. Please check your input')
    exit()

# Define the Region of interest (ROI)
roi = selected_feature.geometry()

# Ask user to select a date range (start and end)
start_date_str = input('Enter start date (YYYY-MM-DD): ')
end_date_str = input('Enter end date (YYYY-MM-DD): ')

# Except error for wrong date parameters
try:
    # Turn date/time string into Python datetime object
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d')
    
    # Date formate being used is YYYY-MM-DD
    # GEE requires the format YYYY-MM-DD 
except ValueError:
    print('Invalid date format. Please enter dates in YYYY-MM-DD format.')
    exit()

# Load the new Harmonized collection (Old collection was deprecated in Spring of 2025)
sentinel_collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                       # Filters collection to only areas in ROI
                       .filterBounds(roi)
                       # Filter images based on date inputs 
                       .filterDate(start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))
                       # Filter out images where cload coverage is 10% or more
                       .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
                       # Applys the ndci calculation to each image found 
                       .map(calculate_ndci)
                       # Apply the land mask function
                       .map(mask_land)
                       # Clip the image to the area arounf the ROI
                       .map(lambda img: img.clip(roi))
                      )

# Count images found and list count 
image_count = sentinel_collection.size().getInfo()

# Print out the number of images found to the user 
print(f'Number of cloud-free images found: {image_count}')

# When images are found
if image_count > 0:

    # Count images
    image_list = sentinel_collection.toList(image_count)

    # Collect image dates 
    dates = [ee.Image(image_list.get(i)).date().format('yyyy-MM-dd').getInfo() for i in range(image_count)]
    # Print available dates to the user
    print("Available dates:", dates)

    # Calculate NDCI values (mean, max, min)
    # Set a threshold (0.2–0.5 for moderate vegetation)
    ndci_threshold = 0.25
    dates_of_interest = []

    # Create a for loopto process GEE images
    for i in range(image_count):
        # Retreave an image and cast it to an ee.Image
        img = ee.Image(image_list.get(i))
        # Select the NDCI band from the image
        ndci_img = img.select('NDCI')
        # Extract metadata (date)
        # Use getInfo() to bring the image into python
        date = img.date().format('yyyy-MM-dd').getInfo()

                # Calculate Mean NDCI for selected lagoon
        mean_stats = ndci_img.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=roi,
            scale=20,
            maxPixels=1e9
        )
        # Extract the mean value and bring it into Python
        mean_val = mean_stats.get('NDCI').getInfo()

        # Calculate Max NDCI for selected lagoon
        max_stats = ndci_img.reduceRegion(
            reducer=ee.Reducer.max(),
            geometry=roi,
            scale=20,
            maxPixels=1e9
        )
        # Extract the max value and bring it into Python
        max_val = max_stats.get('NDCI').getInfo()

        # Calculate Min NDCI for selected lagoon
        min_stats = ndci_img.reduceRegion(
            reducer=ee.Reducer.min(),
            geometry=roi,
            scale=20,
            maxPixels=1e9
        )
        # Extract the min value and bring it into Python
        min_val = min_stats.get('NDCI').getInfo()

        # If mean NDCI is valid, append values to list
        if mean_val is not None:
            dates_of_interest.append({
                'date': date,
                'mean': round(mean_val, 3),
                'max': round(max_val, 3) if max_val is not None else None,
                'min': round(min_val, 3) if min_val is not None else None
            })

    # Print summary with bloom highlight
    if dates_of_interest:
        
        # Loop through all collected NDCI stats
        print("\nDates with NDCI statistics (mean, max, min):")
        for entry in dates_of_interest:
            # Print date and stats, highlight if mean > threshold
            line = f" - {entry['date']}: Mean = {entry['mean']}, Max = {entry['max']}, Min = {entry['min']}"
            if entry['mean'] > ndci_threshold:
                line += "  ← bloom candidate"
            print(line)
    else:
        # Print message if no valid data found
        print("\nNo valid NDCI values found.")

    # Let user view selected image
    while True:
        # Ask user to input a date or exit
        selected_date = input("Enter the date you want to view (or type 'Stop' to exit): ")
        if selected_date.lower() == 'stop':
            break

        # If date is valid, display map
        if selected_date in dates:
            index = dates.index(selected_date)
            selected_image = ee.Image(image_list.get(index)).clip(roi)

            # Create and center interactive map
            Map = geemap.Map()
            Map.centerObject(roi, 10)

            # Basemap can be chaned 
            # For other basemap options and geemap format 
            # visit the geemap website https://geemap.org
            Map.add_basemap('Esri.WorldTopoMap')       # Can change basemap

            # Define visualization parameters
            rgb_vis = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000, 'gamma': 1.4}
            ndci_vis = {
                'bands': ['NDCI'],
                'min': 0, 'max': 0.5,
                'palette': ['blue', 'green', 'yellow', 'orange', 'red']
            }

            # Add RGB, NDCI, and boundary layers to map
            Map.addLayer(selected_image, rgb_vis, f"RGB - {lagoon_name} - {selected_date}")
            Map.addLayer(selected_image.select('NDCI'), ndci_vis, 'NDCI (Algal Bloom Index)')
            Map.addLayer(selected_feature.style(color='black', fillColor='00000000'), {}, 'Lagoon Boundary')

            # If you want to add a title uncomment the text below
            
            # Add a title to the map
            # Map.add_text(
            # text=f"NDCI for {lagoon_name} on {selected_date}",
            # fontsize=14,
            # position="topright",
            # fontcolor="black",
            # bgcolor="white"
                        #)

            def display_ndci_legend():

                # Define NDCI range (clipped to realistic bloom values)
                ndci_min, ndci_max = -0.2, 0.8  

                # Create a custom colormap
                colors = ['blue', 'green', 'yellow', 'orange', 'red']
                cmap = LinearSegmentedColormap.from_list("custom_ndci", colors, N=256)

                norm = plt.Normalize(vmin=ndci_min, vmax=ndci_max)

                # Create the colorbar
                fig, ax = plt.subplots(figsize=(6, 1))
                fig.subplots_adjust(bottom=0.5)

                cb = plt.colorbar(
                    plt.cm.ScalarMappable(norm=norm, cmap=cmap),
                    cax=ax,
                    orientation='horizontal'
                                )

                cb.set_label("NDCI (Algal Bloom Index)", fontsize=12)
                plt.show()

            # Display the map
            display(Map)
            display_ndci_legend()

        else:
            # If date not found, show error
            print("Date not found. Please choose from available options.")
else:
    # If no images found at all, print message
    print("No valid images found for this lagoon and date range.")

Avalible lagoons: ['Krusenstern', 'Kotlik', 'Aukulak', 'Kivalina', 'Ikpek', 'Kupik', 'Aiautak', 'Kemegrak', 'Akoviknak', 'Atosik', 'Mapsorak', 'Pusigrak', 'Singoalik', 'Seppings', 'Tasikpak', 'Pusaluk', 'Tugak', 'Kavrorak', 'Asikpak', 'Imukruk', 'Ipiavik', 'Tsaitsat Angayukangnk', 'Imik', 'Tasaychek', 'Atilagauraq']


Enter an avlible lagoon:  Krusenstern
Enter start date (YYYY-MM-DD):  2020-06-13
Enter end date (YYYY-MM-DD):  2020-06-25


Number of cloud-free images found: 4
Available dates: ['2020-06-14', '2020-06-19', '2020-06-20', '2020-06-24']

Dates with NDCI statistics (mean, max, min):
 - 2020-06-14: Mean = 0.143, Max = 0.328, Min = -0.129
 - 2020-06-19: Mean = 0.184, Max = 0.32, Min = -0.202
 - 2020-06-20: Mean = 0.216, Max = 0.324, Min = -0.055
 - 2020-06-24: Mean = 0.23, Max = 0.293, Min = -0.028
